In [1]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
# from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
# from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,TextIteratorStreamer,LlamaTokenizer
)
# from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

C:\Users\maxme\AppData\Local\Temp\ipykernel_9056\4060503811.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
notebook_login()

In [3]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
        rope_scaling = {"type":"dynamic","factor":2.0}
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [4]:
model, tokenizer = create_model_and_tokenizer()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
pip install https://github.com/jllllll/bitsandbytes-windows-webui/raw/77b2247e73e3a39f8314f1e2e2916c601a733a82/bitsandbytes-0.39.0-py3-none-any.whl --force-reinstall --no-deps

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.41.0
    Uninstalling bitsandbytes-0.41.0:
      Successfully uninstalled bitsandbytes-0.41.0
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'f:\Projects\Hackathon brothers and sisters\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
streamer = TextIteratorStreamer(
            tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True
        )
def summarize(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs ,streamer =streamer,max_new_tokens=512,
                output_attentions=False,
                output_hidden_states=False,repetition_penalty = 1.15)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

In [7]:
df = pd.read_csv(r"data\depression_dataset_reddit_cleaned.csv")
df.rename(columns={"clean_text": "text", "is_depression": "class"}, inplace=True)
df.head(), df.shape

(                                                text  class
 0  we understand that most people who reply immed...      1
 1  welcome to r depression s check in post a plac...      1
 2  anyone else instead of sleeping more when depr...      1
 3  i ve kind of stuffed around a lot in my life d...      1
 4  sleep is my greatest and most comforting escap...      1,
 (7731, 2))

In [8]:
num = 144
example = df["text"][0]
example 

'we understand that most people who reply immediately to an op with an invitation to talk privately mean only to help but this type of response usually lead to either disappointment or disaster it usually work out quite differently here than when you say pm me anytime in a casual social context we have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support others here and flag inappropriate content even more so because we know that so many of you are struggling yourselves we re hard at work behind the scene on more information and resource to make it easier to give and get quality help here this is just a small start our new wiki page explains in detail why it s much better to respond in public comment at least until you ve gotten to know someone it will be maintained at r depression wiki private contact and the full text of the current version is below summary anyone who while acting a a helper invite or accepts private contact i e pm chat

In [9]:
%%time
summary = summarize(model, example)
print(summary)

f:\Projects\Hackathon brothers and sisters\venv\lib\site-packages\transformers\models\llama\modeling_llama.py:728: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


KeyboardInterrupt: 